In [53]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [55]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [56]:
df.drop(['packer_type'],axis=1,inplace=True)

In [57]:
from sklearn.utils import shuffle
df = shuffle(df)

In [58]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [59]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [60]:
df.drop(['index'],axis=1,inplace=True)

In [61]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,1,1,...,0,4,0,0,0.000000,0.000000,40448,7.066267,0,1
1,144,3,4,65535,184,304,4,1,0,1,...,1,1,3,0,6.346469,5.130366,439600,6.745621,1,0
2,144,3,4,65535,184,248,4,1,1,1,...,1,1,3,0,0.000000,0.199479,105472,5.627922,0,1
3,144,3,4,65535,184,240,6,1,1,1,...,1,3,3,0,0.000000,0.000000,80896,6.273655,1,1
4,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,6.042422,0.771466,41051,4.708580,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,128,3,1,1,1,...,1,0,3,0,7.262762,7.959451,336896,7.565592,0,1
5206,144,3,4,65535,184,192,1,1,0,1,...,1,0,1,0,0.000000,0.000000,1536,3.353099,1,0
5207,144,3,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,211456,7.836653,1,1
5208,80,2,4,65535,184,124,4,1,1,1,...,1,0,4,0,5.316944,0.062379,17920,5.425462,0,1


In [62]:
df_5percent=df[0:3647]

In [63]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,1,1,...,0,4,0,0,0.000000,0.000000,40448,7.066267,0,1
1,144,3,4,65535,184,304,4,1,0,1,...,1,1,3,0,6.346469,5.130366,439600,6.745621,1,0
2,144,3,4,65535,184,248,4,1,1,1,...,1,1,3,0,0.000000,0.199479,105472,5.627922,0,1
3,144,3,4,65535,184,240,6,1,1,1,...,1,3,3,0,0.000000,0.000000,80896,6.273655,1,1
4,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,6.042422,0.771466,41051,4.708580,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3642,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,227136,5.225364,1,0
3643,144,3,4,65535,184,184,3,1,1,1,...,1,2,1,1,0.000000,0.000000,1497088,7.998952,0,1
3644,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.247474,1.387610,101376,6.272934,1,0
3645,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,1,5.496309,0.275008,36941,3.945872,1,0


In [64]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1743, 69)
class 1: (1904, 69)


In [65]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [66]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     256.0               4.0   
1      144.0   3.0        4.0     65535.0  184.0     304.0               4.0   
2      144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     240.0               6.0   
4      144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
...      ...   ...        ...         ...    ...       ...               ...   
3642   144.0   3.0        4.0     65535.0  184.0     184.0               1.0   
3643   144.0   3.0        4.0     65535.0  184.0     184.0               3.0   
3644   144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
3645   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
3646   144.0   3.0        4.0     65535.0  184.0     256.0               6.0   

      C

In [67]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3647.0,...,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3647.000000,3.647000e+03,3647.000000,3647.000000,3647.000000
mean,143.983822,8.411845,7.499040,65208.664656,188.216068,222.896627,4.686318,0.990677,0.358377,1.0,...,0.999177,1.383329,3.302989,0.155744,4.960517,2.538943,8.012622e+05,6.366886,0.546202,0.522073
std,292.017536,331.958331,212.733108,4535.869978,291.242387,48.412313,1.960887,0.096116,0.479589,0.0,...,0.028673,1.667681,1.122037,0.362663,2.500518,2.693459,5.535809e+06,1.108885,0.497929,0.499581
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.943013,0.000000,6.092800e+04,5.696455,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.153861,1.535390,1.239040e+05,6.388249,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.511421,4.709391,3.033760e+05,7.284624,1.000000,1.000000
max,17739.000000,20050.000000,12851.000000,65535.000000,17744.000000,648.000000,29.000000,1.000000,1.000000,1.0,...,1.000000,26.000000,7.000000,1.000000,7.999859,7.999607,1.657081e+08,7.999997,1.000000,1.000000


In [68]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [69]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [70]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [71]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      55.625
0.0      44.375
dtype: float64

In [72]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      57.5
0.0      42.5
dtype: float64

In [73]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [74]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [75]:
circuit(weights_init, data[0])

tensor(0.04007183, requires_grad=True)

In [76]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [77]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [78]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [79]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [80]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.08191251, requires_grad=False), tensor(-0.10736493, requires_grad=False)], Y = -1
X = [tensor(-0.08201586, requires_grad=False), tensor(-0.11177992, requires_grad=False)], Y =  1
X = [tensor(-0.08265831, requires_grad=False), tensor(-0.12264407, requires_grad=False)], Y =  1
X = [tensor(-0.08154659, requires_grad=False), tensor(-0.09130603, requires_grad=False)], Y =  1
X = [tensor(-0.08265824, requires_grad=False), tensor(-0.1164524, requires_grad=False)], Y =  1


In [81]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [82]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0803499 | Accuracy: 0.5562500 
Iter:     2 | Cost: 1.0421673 | Accuracy: 0.5562500 
Iter:     3 | Cost: 0.9724289 | Accuracy: 0.5562500 
Iter:     4 | Cost: 0.9766120 | Accuracy: 0.5562500 
Iter:     5 | Cost: 0.9891624 | Accuracy: 0.5562500 
Iter:     6 | Cost: 0.9710722 | Accuracy: 0.5562500 
New best
Iter:     7 | Cost: 0.9733837 | Accuracy: 0.6250000 
Iter:     8 | Cost: 1.0024642 | Accuracy: 0.4437500 
New best
Iter:     9 | Cost: 0.9790324 | Accuracy: 0.6500000 
Iter:    10 | Cost: 0.9682514 | Accuracy: 0.5562500 
Iter:    11 | Cost: 1.0127354 | Accuracy: 0.5562500 
Iter:    12 | Cost: 1.0320029 | Accuracy: 0.5562500 
Iter:    13 | Cost: 0.9944345 | Accuracy: 0.5562500 
Iter:    14 | Cost: 0.9705420 | Accuracy: 0.5687500 
Iter:    15 | Cost: 0.9616639 | Accuracy: 0.5812500 
Iter:    16 | Cost: 0.9585585 | Accuracy: 0.6062500 
Iter:    17 | Cost: 0.9543933 | Accuracy: 0.6000000 
Iter:    18 | Cost: 0.9515093 | Accuracy: 0.5875000 
Iter:    19 | Cost:

In [83]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [84]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.3963114536074466, Accuracy: 50.0%


In [85]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,1.0
2,1.0,-1.0
3,1.0,-1.0
4,1.0,-1.0
5,1.0,1.0
6,1.0,1.0
7,1.0,-1.0
8,1.0,1.0
9,1.0,-1.0


In [86]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         3
         1.0       0.87      0.54      0.67        37

    accuracy                           0.50        40
   macro avg       0.43      0.27      0.33        40
weighted avg       0.80      0.50      0.62        40

0.8695652173913043
0.5405405405405406
0.6666666666666666
0.2702702702702703
